In [2]:
%pip install -q requests
%pip install -q json
import requests
import json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [1]:
base_discovery_url = "https://discovery.nationalarchives.gov.uk/API"

In [3]:
parent_id = "F236668"

##Working through the collections

In Chapter 2 we obtained specific record from discovery API, including the ID of the record as well as that of its parent's. Records are parts of collection, if we want to learn more about the collection or what other records are in the collection we will make use of the parent's ID we retrieved in the last chapter and a new endpoint "/records/v1/children/{parentId}". The parent ID here is case-sensitive, so if we use f236668 instead of F236668 then it won't return any result.

In [4]:
collection_endpoint = "/records/v1/children/"

In [5]:
full_search_url = base_discovery_url + collection_endpoint + parent_id

response = requests.request("GET", full_search_url)

print(json.dumps(response.json(), indent=4))

{
    "assets": [
        {
            "accessConditions": null,
            "catalogueId": 0,
            "citableReference": null,
            "closureCode": null,
            "closureStatus": null,
            "closureType": null,
            "coveringDates": "1681",
            "coveringFromDate": 16810101,
            "coveringToDate": 16811231,
            "scopeContent": {
                "placeNames": [],
                "description": "map",
                "ephemera": null,
                "schema": "<doctype id= \"21\">map</doctype>"
            },
            "digitised": false,
            "heldBy": [
                {
                    "xReferenceId": "A13532511",
                    "xReferenceCode": "54",
                    "xReferenceName": "Canterbury Cathedral Archives",
                    "xReferenceType": null,
                    "xReferenceURL": null,
                    "xReferenceDescription": null,
                    "xReferenceSortWord": null
          

We will check how many records have been returned (the default value) and if there are any more records.

In [6]:
print(len(response.json()['assets']), response.json()['hasMoreAfterLast'])

30 True


We will save this as a new list.

In [7]:
sibling_records = response.json()['assets']

30


To get all the sibling records, we will loop the API call till "hasMoreAfterLast" is true and use the id of the last record to get the records after it.

In [15]:
while(response.json()['hasMoreAfterLast']):
  last_id = response.json()['assets'][-1]["id"]
  full_search_url = base_discovery_url + collection_endpoint + parent_id + "?direction=NEXT&batchStartRecordId=" + last_id
  response = requests.request("GET", full_search_url)
  sibling_records = sibling_records + response.json()['assets'][1:]
print(len(sibling_records))

57


We can also check if there is any record down the heirarchy by using the id of the record as the parent ID

In [17]:
parent_id =  "N14309820"

In [18]:
full_search_url = base_discovery_url + collection_endpoint + parent_id

response = requests.request("GET", full_search_url)

print(json.dumps(response.json(), indent=4))

{
    "assets": [],
    "hasMoreAfterLast": false,
    "hasMoreBeforeFirst": false
}


There aren't any.